In [3]:
!ls model-examples/sbml-core

BIOMD0000000001.omex
BIOMD0000000001_url.xml
BIOMD0000000002.omex
BIOMD0000000005
BIOMD0000000005.omex
BIOMD0000000005_url.xml
BIOMD0000000010.omex
BIOMD0000000010_url.xml
BIOMD0000000040.omex
BIOMD0000000488_url.xml
BIOMD0000000644.omex
BorisEJB.xml
Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations
Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations.omex
Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-Fehlberg
Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-Fehlberg.omex
Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous
Ciliberto-J-Cell-Biol-2003-morphogenesis-checkpoint-continuous.omex
Edelstein-Biol-Cybern-1996-Nicotinic-excitation
Edelstein-Biol-Cybern-1996-Nicotinic-excitation.omex
Elowitz-Nature-2000-Repressilator
Elowitz-Nature-2000-Repressilator.omex
Parmar-BMC-Syst-Biol-2017-iron-distribution
Parmar-BMC-Syst-Biol-2017-iron-distribution.omex
Szymanska-J-Theor-Biol-2009-HSP-synthesis
Szymanska-J-Theor-Biol-2009-HSP-synthesis.omex
Tomida-EMBO-

In [13]:
import docker

# Initialize a Docker client
client = docker.from_env()

# Define the image and the paths
image = "ghcr.io/biosimulators/vcell:7.6.0.40"
input_file = "model-examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex"
output_dir = "test_outputs"
client.images.pull(image)

<Image: 'ghcr.io/biosimulators/vcell:7.6.0.40'>

In [15]:
try:
    print(f"Running container {image}...")

    # Run the container
    container = client.containers.run(
        image,  # The image name
        f"-i {input_file} -o {output_dir}",  # CLI arguments to pass to the container
        # volumes={
            # '/path/to': {'bind': '/path/to', 'mode': 'rw'},  # Mount the host path to container
        # },
        # detach=True,  # Run in the background (optional)
        remove=True,  # Automatically remove the container after it exits
    )

    # Optionally wait for the container to finish and get logs
    container.wait()
    logs = container.logs()
    print(logs.decode('utf-8'))

except docker.errors.ContainerError as e:
    print(f"Error running the container: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Running container ghcr.io/biosimulators/vcell:7.6.0.40...
Error running the container: Command '-i model-examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex -o test_outputs' in image 'ghcr.io/biosimulators/vcell:7.6.0.40' returned non-zero exit status 1: b''


In [8]:
client.images

In [9]:
dir(client.images)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'build',
 'client',
 'create',
 'get',
 'get_registry_data',
 'list',
 'load',
 'model',
 'prepare_model',
 'prune',
 'prune_builds',
 'pull',
 'push',
 'remove',
 'search']

In [ ]:
docker run \
  --tty \
  --rm \
  --mount type=bind,source={ model-examples/sbml-core },target=/tmp/project,readonly \
  --mount type=bind,source={ test_outputs },target=/tmp/results \
  ghcr.io/biosimulators/vcell:7.6.0.40 \
    --archive /tmp/project/{ Elowitz-Nature-2000-Repressilator.omex } \
    --out-dir /tmp/results

In [1]:
import os
import tempfile

import h5py
import docker


def run_biosimulators_container(simulator: str, archive_fp: str, output_dest: str = None):
    pwd = os.getcwd()
    client = docker.from_env()
    input_dir = os.path.abspath(os.path.dirname(archive_fp))
    archive_filename = archive_fp.split('/')[-1]
    output_dir = os.path.abspath(output_dest or tempfile.mkdtemp())
    
    volumes = {
        input_dir: {"bind": "/tmp/project", "mode": "ro"},
        output_dir: {"bind": "/tmp/results", "mode": "rw"}
    }
    
    try:
        container = client.containers.run(
            image=f"ghcr.io/biosimulators/{simulator}:latest",
            command=f"--archive /tmp/project/{archive_filename} --out-dir /tmp/results",
            volumes=volumes,
            tty=True,
            remove=True  # auto-remove
        )
        return f'{output_dir}/reports.h5'
    except docker.errors.ContainerError as e:
        print(f"Error: {e}")


def read_report_outputs(report_file_path) -> dict:
    """Read the outputs from all species in the given report file from biosimulations output.
        Args:
            report_file_path (str): The path to the simulation.sedml/report.h5 HDF5 file.
    """
    # TODO: implement auto gen from run id here.
    outputs = []
    with h5py.File(report_file_path, 'r') as f:
        k = list(f.keys())
        group_path = k[0] + '/report'
        if group_path in f:
            group = f[group_path]
            dataset_labels = group.attrs['sedmlDataSetLabels']
            for label in dataset_labels:
                dataset_index = list(dataset_labels).index(label)
                data = group[()]
                specific_data = data[dataset_index]
                output = {'dataset_label': label, 'data': specific_data}
                outputs.append(output)
            return {'report_path': report_file_path, 'data': outputs}
        else:
            return {'report_path': report_file_path, 'data': f"Group '{group_path}' not found in the file."}


In [2]:
input_file = "model-examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex"
input_file2 = "model-examples/sbml-core/new-simulators.omex"
output_dir = "test_outputs"
report_fp = run_biosimulators_container('vcell', input_file2, output_dir)

In [3]:
outs = read_report_outputs(report_fp)

In [4]:
outs

{'report_path': '/Users/alexanderpatrie/Desktop/repos/bio-check/test_outputs/reports.h5',
 'data': [{'dataset_label': b'Time',
   'data': array([0.000e+00, 5.000e-01, 1.000e+00, ..., 9.990e+02, 9.995e+02,
          1.000e+03])},
  {'dataset_label': b'Dynamics of species "MAPK"',
   'data': array([280.        , 279.98214651, 279.96583953, ...,   3.37482283,
            3.40534938,   3.4361884 ])},
  {'dataset_label': b'Dynamics of species "MAPK_P"',
   'data': array([10.        , 10.06784912, 10.13413728, ..., 10.08817912,
          10.14192854, 10.19598429])},
  {'dataset_label': b'Dynamics of species "MAPK_PP"',
   'data': array([ 10.        ,   9.95000437,   9.90002319, ..., 286.53699805,
          286.45272207, 286.36782731])},
  {'dataset_label': b'Dynamics of species "MKK"',
   'data': array([280.        , 280.02891734, 280.05296596, ..., 204.36255258,
          204.63137181, 204.9001477 ])},
  {'dataset_label': b'Dynamics of species "MKK_P"',
   'data': array([10.        , 10.069

In [ ]:
pwd = os.getcwd()
client = docker.from_env()
input_dir = os.path.dirname(archive_fp)
volumes = {
    f"{pwd}/model-examples/sbml-core": {"bind": "/tmp/project", "mode": "ro"},
    f"{pwd}/test_outputs": {"bind": "/tmp/results", "mode": "rw"}
}
try:
    container = client.containers.run(
        image="ghcr.io/biosimulators/vcell:7.6.0.40",
        command="--archive /tmp/project/Elowitz-Nature-2000-Repressilator.omex --out-dir /tmp/results",
        volumes=volumes,
        tty=True,
        remove=True  # auto-remove
    )
    return None
except docker.errors.ContainerError as e:
    print(f"Error: {e}")

In [4]:
import os 

input_file = "model-examples/sbml-core/Elowitz-Nature-2000-Repressilator.omex"

os.path.abspath(os.path.dirname(input_file))

'/Users/alexanderpatrie/Desktop/repos/bio-check/model-examples/sbml-core'